In [4]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## Setup

In [9]:
ingredients_description_df = pd.read_csv("ingredientsList.csv")
reactions_df = pd.read_csv("hf://datasets/misbahbagwan/Skincare-Reaction/skincare_data.csv")
product_ingredients_df = pd.read_csv("ulta_joined.csv", index_col = 0)

In [6]:
reactions_df

,Age,Melanin Concentration,Collagen Amount,Epidermis Thickness,Hydration Level,Skin Type,Ingredient,Concentration,Neutral Reaction,Positive Reaction,Brightening Reaction,Irritation Reaction,Negative Reaction
0,56,24.699074,53.752102,0.328596,63.223815,Sensitive,Salicylic Acid,10.831598,0.0,0.0,0.0,0.0,0.0
1,46,37.731765,62.010655,0.212318,58.360504,Sensitive,Azelaic Acid,7.275297,0.0,0.0,0.0,0.0,0.0
2,32,5.145802,73.572630,NaN,39.012023,Normal,Vitamin C,15.137619,0.0,1.0,1.0,0.0,0.0
3,60,26.824068,50.448681,0.122448,35.513165,Dry,Glycolic Acid,1.334878,0.0,0.0,0.0,0.0,0.0
4,25,18.941078,79.353617,0.411056,4.524919,Oily,Azelaic Acid,2.457557,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,50,13.300964,58.707234,0.130408,27.944531,Sensitive,Mandelic Acid,18.864892,0.0,0.0,0.0,0.0,0.0
996,21,7.461889,82.657038,0.068175,78.250014,Normal,Niacinamide,23.895423,0.0,0.0,0.0,0.0,0.0
997,39,6.359988,67.791642,0.153165,88.650253,Oily,Niacinamide,19.325029,0.0,0.0,0.0,0.0,0.0
998,19,43.233020,84.308749,0.196557,39.864249,Dry,Salicylic Acid,21.075046,0.0,0.0,0.0,0.0,0.0


In [7]:
ingredients_description_df

,name,scientific_name,short_description,what_is_it,what_does_it_do,who_is_it_good_for,who_should_avoid,url
0,Alpha-Glucan Oligosaccharide,NaN,Alpha-glucan oligosaccharide is in a class of ...,Prebiotics are a type of non-digestible dietar...,Prebiotics offer benefits such as:\r\n\r\n- He...,"[' ', 'Acne', ' ', 'Blackheads', ' ', 'Redness...","[' ', 'Related Allergy']",https://renude.co/ingredients/alpha-glucan-oli...
1,Aloe Vera,NaN,"Aloe vera, also appear on ingredients lists as...",Aloe vera is a skincare ingredient derived fro...,Aloe vera offers multiple benefits for the ski...,"[' ', 'Dry and dehydrated skin', ' ', 'Impaire...","[' ', 'Related Allergy']",https://renude.co/ingredients/aloe-vera
2,Allantoin,NaN,"Allantoin occurs naturally in the body, but ca...",Allantoin is a skincare ingredient derived fro...,"Allantoin is a calming, anti-inflammatory, and...","[' ', 'Fine Lines', ' ', 'Impaired skin barrie...","[' ', 'Related Allergy']",https://renude.co/ingredients/allantoin
3,Algin,NaN,"Algin, also known as sodium alginate, is a lar...",An extract from brown seaweed used as hydratin...,"In skincare products, it is used for its excel...","[' ', 'Dry and dehydrated skin', ' ', 'Fine Li...","[' ', 'Related Allergy']",https://renude.co/ingredients/algin
4,Algae Extract,NaN,"It is essentially an underwater plant, designe...",An incredibly interesting natural ingredient s...,Algae extracts are multifunctional ingredients...,"[' ', 'Dry and dehydrated skin', ' ', 'Fine Li...","[' ', 'Related Allergy']",https://renude.co/ingredients/algae-extract
...,...,...,...,...,...,...,...,...
243,Zinc Sulfate,NaN,The jury is still out on the efficacy of this ...,An antibacterial ingredient used in skincare.,"Zinc salts such as zinc gluconate, zinc PCA an...","[' ', 'Anyone']","[' ', 'Related Allergy']",https://renude.co/ingredients/zinc-sulfate
244,Zinc PCA,NaN,Zinc PCA is a component of the skin’s natural ...,Zinc PCA is known to regulate sebum production...,"Zinc salts such as zinc gluconate, zinc PCA an...","[' ', 'Acne', ' ', 'Blackheads', ' ', 'Enlarge...","[' ', 'Related Allergy']",https://renude.co/ingredients/zinc-pca
245,Neoglucosamine,NaN,A patented non-amino acid sugar which gently e...,Neoglucosamine is a patented skincare ingredie...,Neoglucosamine offers benefits such as:\n- Sup...,"[' ', 'Blackheads', ' ', 'Elasticity', ' ', 'E...","[' ', 'Related Allergy', ' ', 'Impaired skin b...",https://renude.co/ingredients/neoglucosamine
246,Taurine,NaN,"In skin care, taurine is a powerful form of an...",Taurine is an amino acid that’s naturally prod...,Helps support the skin barrier and protect aga...,"[' ', 'Dry and dehydrated skin', ' ', 'Radianc...","[' ', 'Related Allergy']",https://renude.co/ingredients/taurine


----

In [10]:
product_ingredients_df

,Product Names,Ingredients,Prices,Link
0,D-Bronzi Anti-Pollution Sunshine Drops,"Water/Aqua/Eau, Hydrogenated Polyisobutene, Gl...",$38.00,https://www.ulta.com/p/d-bronzi-anti-pollution...
1,Moisture Surge 100H Auto-Replenishing Hydrator...,"Water\Aqua\Eau, Dimethicone, Butylene Glycol, ...",$29.00,https://www.ulta.com/p/moisture-surge-100h-aut...
2,Glass Skin Refining Serum,"Water (Aqua), Propanediol, Butylene Glycol, Ni...",$59.00,https://www.ulta.com/p/glass-skin-refining-ser...
3,Hyaluronic Acid 2% + B5 Hydrating Serum,"Aqua (Water), Sodium Hyaluronate, Pentylene Gl...",$9.90,https://www.ulta.com/p/hyaluronic-acid-2-b5-hy...
4,Glycolic Acid 7% Exfoliating Toning Solution,"Aqua (Water), Glycolic Acid, Rosa Damascena Fl...",$13.00,https://www.ulta.com/p/glycolic-acid-7-exfolia...
...,...,...,...,...
4210,Pep Start Daily UV Protector Broad Spectrum SP...,"Active: Titanium Dioxide 6.3%, Zinc Oxide 4.0%...",$27.00,https://www.ulta.com/p/pep-start-daily-uv-prot...
4213,Clinique For Men Anti-Fatigue Hydrating Concen...,"Active: Avobenzone 3.0%, Octisalate 5.0%<br/>I...",$60.00,https://www.ulta.com/p/clinique-men-anti-fatig...
4214,Mousse Radiance Clarifying Self-Foaming Cleanser,"Aqua / Water / Eau, Sorbitol, Glycerin, Sodium...",$32.00,https://www.ulta.com/p/mousse-radiance-clarify...
4215,Age Smart Overnight Repair Serum,"Helianthus Annuus (Sunflower) Seed Oil, Argani...",NaN,https://www.ulta.com/p/age-smart-overnight-rep...


What I want this to do
1. user answers Age, Skin Type
2. 
